## Task 2

In [26]:
#rozwiazanie na bazie deleted interpolation
#model bazujacy na tagach jak ma dzialac?
# p(t_i | t_(i-1))

##Oblicze najpierw parametry i po prostu 
import random
import re

prob = 0.8 #prawdopodobienstwo, ze bedzie to w K1

#wczytaj tagi


bigrams_tags = {}
bigrams_words = {}

test = {}

for line in open("../poleval_corpus/polish_corpora.txt"):
    prev = "<BOS>"
    line = re.sub('([.,!?()])', r' \1 ', line)
    line = re.sub('\s{2,}', ' ', line)
    for word in line.lower().split() + ["<EOS>"]:

        prev_tag = tags_from_words.get(prev, prev)
        word_tag = tags_from_words.get(word, word)

        if random.random() > prob:
            test[(prev, word)] = test.get((prev, word), 0) + 1
            continue

        if prev not in bigrams_words:
            bigrams_words[prev] = {}
        bigrams_words[prev][word] = bigrams_words[prev].get(word, 0) + 1

        if prev_tag not in bigrams_tags:
            bigrams_tags[prev_tag] = {}
        bigrams_tags[prev_tag][word_tag] = bigrams_tags[prev_tag].get(word_tag, 0) + 1
        prev = word
    
   


In [27]:

w1 = w2 = 0

print(len(test))

i = 1;

for prev, word in test:
    
    prev_tag = tags_from_words.get(prev, prev)
    word_tag = tags_from_words.get(word, word)
    
    if prev not in bigrams_words:
        words_prob = 0
    else:
        if "#SUM#" not in bigrams_words[prev]:
            bigrams_words[prev]["#SUM#"] = sum(bigrams_words[prev].values())
        words_prob = bigrams_words[prev].get(word, 0) / bigrams_words[prev]["#SUM#"]
    
    if prev_tag not in bigrams_tags:
        tags_prob = 0
    else:
        if "#SUM#" not in bigrams_tags[prev_tag]:
            bigrams_tags[prev_tag]["#SUM#"] = sum(bigrams_tags[prev_tag].values())
        tags_prob = bigrams_tags[prev_tag].get(word_tag, 0) / bigrams_tags[prev_tag]["#SUM#"]
    
    count = test[(prev, word)]
    
    if words_prob < tags_prob:
        w2 += count
    else:
        w1 += count
     
    if i%100000 == 0:
        print(w1, w2)
    
print(w1, w2)
print("for words {}".format(w1/(w1+w2))) # 0.34902408313672256
print("for tags {}".format(w2/(w1+w2))) # 0.6509759168632775



21627812
33736552 62923116
for words 0.34902408313672256
for tags 0.6509759168632775


for words 0.34902408313672256  
for tags  0.6509759168632775

In [4]:


def jackard_coefficient(a,b):
    a = list(zip(["<BOS>"] + a, a + ["<EOS>"]))
    b = list(zip(["<BOS>"] + b, b + ["<EOS>"]))
    
    both = 0
    for pair in a:
        if pair in b:
            both += 1
    
    return both/(len(a)+len(b)-both)
    

0.14285714285714285

In [5]:

tags_from_words = {}
for line in open("../resources/supertags.txt"):
    word, tag = line.split()
    tags_from_words[word] = tag
    


In [28]:

bigrams_tags = {}
bigrams_words = {}

test = []

for line in open("../poleval_corpus/poleval_2grams.txt"):
    count, prev, word = line.split()
    count = int(count)
    prev_tag = tags_from_words.get(prev, prev)
    word_tag = tags_from_words.get(word, word)

    
    if prev not in bigrams_words:
        bigrams_words[prev] = {}
    bigrams_words[prev][word] = count
    
    if prev_tag not in bigrams_tags:
        bigrams_tags[prev_tag] = {}
    bigrams_tags[prev_tag][word_tag] = bigrams_tags[prev_tag].get(word_tag, 0) + count
    


In [29]:
for prev in bigrams_words:
    bigrams_words[prev]["#SUM#"] = sum(bigrams_words[prev].values())
    
for prev_tag in bigrams_tags:
    bigrams_tags[prev_tag]["#SUM#"] = sum(bigrams_tags[prev_tag].values())
    

In [49]:

wsp1 = 0.34902408313672256
wsp2 = 0.6509759168632775

def score(sentence):
    prev = "<BOS>"
    prev_tag = tags_from_words.get(prev, prev)
    result = 0.
    for word in sentence + ["<EOS>"]:
        #print(prev, word)
        
        word_tag = tags_from_words.get(word, word)
        
        if prev in bigrams_words:
            result += wsp1*bigrams_words[prev].get(word, 0)/bigrams_words[prev]["#SUM#"]
        if prev_tag in bigrams_tags:
            result += wsp2*bigrams_tags[prev_tag].get(word_tag, 0)/bigrams_tags[prev_tag]["#SUM#"]
        
        prev = word
        prev_tag = word_tag
        
    return result/len(sentence)

print(score("jak dobre jest to zdanie moje ?".split()))

0.16313807457953383


In [66]:
import itertools

def get_best_sentence(sentence):
    best_val = -1
    best = None
    
    for perm in itertools.permutations(sentence):
        perm_val = score(list(perm))
        #print(" ".join(perm), perm_val)
        if perm_val > best_val:
            best = perm
            best_val = perm_val
    return best


get_best_sentence("naturalny zdania szyk nie jest to".split())


('nie', 'jest', 'to', 'naturalny', 'szyk', 'zdania')

In [ ]:

result = 0
test_cases = 0
interpunkcja = '([.,!?()])'
lines = 1

for line in open("../poleval_corpus/polish_corpora.txt"):
    line = re.sub('([.,!?()])', r' \1 ', line)
    line = re.sub('\s{2,}', ' ', line)
    sentence = line.lower().split()
    
    if lines % 100000 == 0:
        print(result, test_cases)
    lines += 1
    
    if len(sentence) < 4 or len(sentence) > 8:
       # print(len(sentence))
        continue
    
    good = True
    
    for word in sentence:
        if word not in tags_from_words and word not in interpunkcja:
         #   print(word)
            good = False
            break
    
    if not good:
        continue
    
    test_cases += 1
    best = get_best_sentence(sentence)
    result += jackard_coefficient(sentence, list(best))
    #print(" ".join(sentence))
    #print(" ".join(best))
     


2270.8841753018673 4953
4422.622743433007 9737
6573.995672954107 14413
8820.193741551924 19289
11046.061392365496 24087


KeyboardInterrupt: 

In [74]:
print(result/test_cases)

0.4585344020934776
